In [ ]:
import fast

# importer = fast.ImageFileImporter\
#     .create(fast.Config.getDocumentationPath() + '/images/FAST_logo_square.png')

# renderer = fast.ImageRenderer.create()\
#     .connect(importer)

# fast.SimpleWindow2D.create()\
#     .connect(renderer)\
#     .run()

In [ ]:
# # Render a slide
# import fast

# fast.downloadTestDataIfNotExists()  # This will download the test data needed to run the example

# importer = fast.WholeSlideImageImporter.create(
#     fast.Config.getTestDataPath() + "WSI/CMU-1.svs"
# )

# renderer = fast.ImagePyramidRenderer.create().connect(importer)

# fast.SimpleWindow2D.create().connect(renderer).run()

In [ ]:
tiff_path = "../data/images/01lRJdxbiV_a.tif"

In [ ]:
# Extract patches of tissue

import fast
import matplotlib.pyplot as plt

# This will download the test data needed to run the example
fast.downloadTestDataIfNotExists()

importer = fast.WholeSlideImageImporter.create(
    tiff_path
    # fast.Config.getTestDataPath() + "WSI/CMU-1.svs"
)

tissueSegmentation = fast.TissueSegmentation.create().connect(importer)

patchGenerator = (
    fast.PatchGenerator.create(1024, 1024, level=1, overlapPercent=0.0)
    .connect(0, importer)
    .connect(1, tissueSegmentation)
)

# Create a 3x3 subplot for every set of 9 patches
N = 9
patch_list = []
for patch in fast.DataStream(patchGenerator):
    patch_list.append(patch)
    if len(patch_list) == N**2:
        # Display the 9 last patches
        f, axes = plt.subplots(N, N, figsize=(10, 10))
        for i in range(N):
            for j in range(N):
                axes[i, j].imshow(patch_list[i + j * N])
        plt.show()
        # patch_list.clear()
        break

In [ ]:
# We can convert an image to array directly
import numpy as np

np.array(patch_list[0]).shape

In [ ]:
importer = fast.WholeSlideImageImporter.create(tiff_path)

tissueSegmentation = fast.TissueSegmentation.create().connect(importer)

patchGenerator = (
    fast.PatchGenerator.create(1024, 1024, level=3, overlapPercent=0.0).connect(
        0, importer
    )
    # .connect(1, tissueSegmentation)
)

# # Create a 3x3 subplot for every set of 9 patches
# patch_list = list(fast.DataStream(patchGenerator))
# len(patch_list)

# Convert patches back into a slide
stitcher = fast.PatchStitcher.create().connect(patchGenerator)

In [ ]:
importer = fast.WholeSlideImageImporter.create(tiff_path)

# Do tissue segmentation to select patches from tissue only
tissueSegmentation = fast.TissueSegmentation.create().connect(importer)

patchGenerator = (
    fast.PatchGenerator.create(512, 512, level=2, overlapPercent=0.25)
    .connect(0, importer)
    .connect(1, tissueSegmentation)
)

for i, patch in enumerate(fast.DataStream(patchGenerator)):
    fast.ImageExporter.create(f"patch_{i}.jpg").connect(patch).run()

In [ ]:
# Renderer
import fast

importer = fast.WholeSlideImageImporter.create(tiff_path)

tissueSegmentation = fast.TissueSegmentation.create().connect(importer)

renderer = fast.ImagePyramidRenderer.create().connect(importer)

segmentationRenderer = fast.SegmentationRenderer.create().connect(tissueSegmentation)

fast.SimpleWindow2D.create().connect(renderer).connect(segmentationRenderer).run()

# Coordinate check

Check coordinates of the patch given by FAST, to see if they match with the expected region of the slide.

In [ ]:
from pathlib import Path

import fast
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

tiff_img = "../data/images/2qj5MlLLBT_a.tif"


# Run importer and get data
imagePyramid = fast.WholeSlideImageImporter.create(tiff_img).runAndGetOutputData()

# tile = "../processed/2qj5MlLLBT_a/patches/26_588_6656_150528_0_256_256.jpg"
tile = "../processed/2qj5MlLLBT_a/patches/26_592_6656_151552_0_256_256.jpg"

list_info = [int(x) for x in Path(tile).stem.split("_")]
patch_id_x, patch_id_y, x, y, level, w, h = list_info

# Extract specific patch at highest resolution
access = imagePyramid.getAccess(fast.ACCESS_READ)
# level = 0
# x = 12000
# y = 14000
# w = 1024
# h = 1024
image = access.getPatchAsImage(level, x, y, w, h)

# Convert FAST image to numpy ndarray and plot
image = np.asarray(image)


# Load the JPG file using Pillow
with Image.open(tile) as img:
    jpg_image = np.array(img)

# Create a figure with 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plot the extracted patch image
axs[0].imshow(image)
axs[0].axis("off")  # Hide axes
axs[0].set_title("Extracted Patch")

# Plot the JPG image
axs[1].imshow(jpg_image)
axs[1].axis("off")  # Hide axes
axs[1].set_title("JPG Image")

# Show the plot
plt.tight_layout()
plt.show()